In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style for publication-quality plots
plt.style.use('seaborn-v0_8-paper')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['xtick.labelsize'] = 9
plt.rcParams['ytick.labelsize'] = 9
plt.rcParams['legend.fontsize'] = 9

# Create output directory
output_dir = Path('presentation_plots')
output_dir.mkdir(exist_ok=True)

print("✅ Setup complete")

## 1. Next Cow Prediction Results

In [ ]:
# Load all next cow prediction results
experiments = {
    'Pure MLP': 'pure_mlp_next_cow_output',
    'MLP + Logic': 'mlp_next_cow_output',
    'Logic Only': 'logic_only_onehot_lr01_3000ep',
    'One-Hot MLP': 'onehot_mlp_output',
}

results = {}
for name, path in experiments.items():
    exp_path = Path(path)
    if exp_path.exists():
        # Load config
        with open(exp_path / 'config.json', 'r') as f:
            config = json.load(f)
        
        # Load history if exists
        history_file = exp_path / 'history.json'
        if history_file.exists():
            with open(history_file, 'r') as f:
                history = json.load(f)
            results[name] = {'config': config, 'history': history}
        else:
            results[name] = {'config': config, 'history': None}
            print(f"⚠️  No history found for {name}")
    else:
        print(f"❌ {name} not found at {path}")

print(f"\n✅ Loaded {len(results)} experiments")
for name in results.keys():
    print(f"   - {name}")

In [ ]:
# Plot 1: Training curves comparison
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
fig.suptitle('Next Cow Prediction: Training Curves Comparison', fontsize=14, fontweight='bold')

metrics = [
    ('train_loss', 'Training Loss', axes[0, 0]),
    ('val_loss', 'Validation Loss', axes[0, 1]),
    ('train_acc', 'Training Accuracy', axes[1, 0]),
    ('val_acc', 'Validation Accuracy', axes[1, 1])
]

for metric_key, title, ax in metrics:
    for name, data in results.items():
        if data['history'] and metric_key in data['history']:
            values = data['history'][metric_key]
            # Subsample if too many epochs
            if len(values) > 500:
                step = len(values) // 500
                values = values[::step]
            ax.plot(values, label=name, linewidth=1.5, alpha=0.8)
    
    ax.set_xlabel('Epoch')
    ax.set_ylabel(title)
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'next_cow_training_curves.png', bbox_inches='tight')
plt.savefig(output_dir / 'next_cow_training_curves.pdf', bbox_inches='tight')
plt.show()

print("✅ Saved: next_cow_training_curves.png/.pdf")

In [ ]:
# Plot 2: Final performance comparison
performance_data = []
for name, data in results.items():
    if data['history']:
        # Get best validation metrics
        best_val_acc = max(data['history']['val_acc']) if 'val_acc' in data['history'] else 0
        best_val_loss = min(data['history']['val_loss']) if 'val_loss' in data['history'] else 0
        final_train_acc = data['history']['train_acc'][-1] if 'train_acc' in data['history'] else 0
        
        performance_data.append({
            'Model': name,
            'Best Val Accuracy': best_val_acc,
            'Best Val Loss': best_val_loss,
            'Final Train Accuracy': final_train_acc
        })

df_perf = pd.DataFrame(performance_data)

# Create bar plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
fig.suptitle('Next Cow Prediction: Performance Comparison', fontsize=14, fontweight='bold')

# Accuracy comparison
x = np.arange(len(df_perf))
width = 0.35
axes[0].bar(x - width/2, df_perf['Best Val Accuracy'], width, label='Val Accuracy', alpha=0.8)
axes[0].bar(x + width/2, df_perf['Final Train Accuracy'], width, label='Train Accuracy', alpha=0.8)
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Best Accuracy')
axes[0].set_xticks(x)
axes[0].set_xticklabels(df_perf['Model'], rotation=45, ha='right')
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Loss comparison
axes[1].bar(df_perf['Model'], df_perf['Best Val Loss'], alpha=0.8)
axes[1].set_ylabel('Loss')
axes[1].set_title('Best Validation Loss')
axes[1].set_xticklabels(df_perf['Model'], rotation=45, ha='right')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig(output_dir / 'next_cow_performance.png', bbox_inches='tight')
plt.savefig(output_dir / 'next_cow_performance.pdf', bbox_inches='tight')
plt.show()

# Save table
df_perf.to_csv(output_dir / 'next_cow_performance.csv', index=False)
print("\n📊 Performance Summary:")
print(df_perf.to_string(index=False))
print("\n✅ Saved: next_cow_performance.png/.pdf/.csv")

## 2. Graph Prediction Results

In [ ]:
# Load graph prediction results
graph_dirs = ['graph_prediction_out', 'graph_prediction_out_bs512']

graph_results = {}
for gdir in graph_dirs:
    exp_path = Path(gdir)
    if exp_path.exists():
        # Look for result files
        result_files = list(exp_path.glob('**/metrics*.json')) + list(exp_path.glob('**/results*.json'))
        if result_files:
            print(f"Found {len(result_files)} result files in {gdir}")
            graph_results[gdir] = result_files

print(f"\n✅ Found {len(graph_results)} graph prediction directories")

## 3. Temporal Graph Visualization

In [ ]:
# Load temporal graph statistics
experiments_data = {
    'Brush': 'brush_experiment',
    'Water Spot': 'water_spot_experiment',
    'Lactation': 'lactation_experiment'
}

temporal_stats = {}
for name, exp_dir in experiments_data.items():
    exp_path = Path(exp_dir)
    if exp_path.exists():
        # Find summary files
        summary_files = list(exp_path.glob('*_summary.csv'))
        if summary_files:
            df = pd.read_csv(summary_files[0])
            temporal_stats[name] = df
            print(f"✅ Loaded {name}: {len(df)} snapshots")

print(f"\n✅ Loaded {len(temporal_stats)} experiment datasets")

In [ ]:
# Plot temporal graph evolution
if temporal_stats:
    fig, axes = plt.subplots(len(temporal_stats), 2, figsize=(12, 4*len(temporal_stats)))
    if len(temporal_stats) == 1:
        axes = axes.reshape(1, -1)
    
    fig.suptitle('Temporal Graph Evolution by Experiment', fontsize=14, fontweight='bold')
    
    for idx, (name, df) in enumerate(temporal_stats.items()):
        # Plot edges over time
        if 'num_edges' in df.columns:
            axes[idx, 0].plot(df['num_edges'], linewidth=1.5)
            axes[idx, 0].set_title(f'{name}: Number of Edges')
            axes[idx, 0].set_xlabel('Time Window')
            axes[idx, 0].set_ylabel('Number of Edges')
            axes[idx, 0].grid(True, alpha=0.3)
        
        # Plot nodes over time
        if 'num_nodes' in df.columns:
            axes[idx, 1].plot(df['num_nodes'], linewidth=1.5, color='orange')
            axes[idx, 1].set_title(f'{name}: Number of Active Nodes')
            axes[idx, 1].set_xlabel('Time Window')
            axes[idx, 1].set_ylabel('Number of Nodes')
            axes[idx, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_dir / 'temporal_graph_evolution.png', bbox_inches='tight')
    plt.savefig(output_dir / 'temporal_graph_evolution.pdf', bbox_inches='tight')
    plt.show()
    
    print("✅ Saved: temporal_graph_evolution.png/.pdf")

## 4. CowBERT Analysis

In [ ]:
# Check if cowbert results exist
cowbert_paths = [
    'results/cow_contrastive_out',
    'outputs'
]

cowbert_data = {}
for path in cowbert_paths:
    path_obj = Path(path)
    if path_obj.exists():
        # Look for result files
        result_files = list(path_obj.glob('**/*.json')) + list(path_obj.glob('**/*.csv'))
        if result_files:
            print(f"Found {len(result_files)} files in {path}")
            cowbert_data[path] = result_files[:10]  # Show first 10
            for f in result_files[:5]:
                print(f"  - {f.name}")

print(f"\n✅ Found CowBERT data in {len(cowbert_data)} directories")

## 5. Summary Table

In [ ]:
# Create comprehensive summary table
summary_data = []

# Next Cow Prediction models
for name, data in results.items():
    if data['history']:
        summary_data.append({
            'Task': 'Next Cow Prediction',
            'Model': name,
            'Best Val Acc': f"{max(data['history']['val_acc']):.4f}" if 'val_acc' in data['history'] else 'N/A',
            'Parameters': data['config'].get('num_parameters', 'N/A'),
            'Architecture': data['config'].get('architecture', 'N/A')
        })

df_summary = pd.DataFrame(summary_data)

print("\n" + "="*80)
print("EXPERIMENT SUMMARY")
print("="*80)
print(df_summary.to_string(index=False))
print("="*80)

# Save summary
df_summary.to_csv(output_dir / 'experiment_summary.csv', index=False)
print("\n✅ Saved: experiment_summary.csv")

## 6. Export All Plots

In [ ]:
# List all generated plots
print("\n" + "="*80)
print("GENERATED PRESENTATION PLOTS")
print("="*80)
print(f"\nOutput directory: {output_dir.absolute()}")
print("\nGenerated files:")

for file in sorted(output_dir.glob('*')):
    size = file.stat().st_size / 1024  # KB
    print(f"  📊 {file.name:50s} {size:>8.1f} KB")

print("\n" + "="*80)
print("✅ ALL PLOTS GENERATED SUCCESSFULLY")
print("="*80)